In [ ]:
import osmium as osm
import pandas as pd
import geopandas
import os.path
import shapely.wkb as wkblib
wkbfab = osm.geom.WKBFactory()
import numpy as np
import shapely.geometry

from sklearn import tree
import graphviz
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn import preprocessing
import jenkspy
from sklearn.metrics import PrecisionRecallDisplay

In [ ]:
class AreaHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.num_polys = 0
        self.osm_data = []
        self.roads_data = []
        self.landuse_data = []
        self.amenity_data = []
        
    def count_polygons(self):
        self.num_polys += 1
        
    def way(self, w):
        try:
            wkb = wkbfab.create_linestring(w)
            line = wkblib.loads(wkb, hex=True)
            for tag in w.tags:
                if (tag.k=='highway'):
                    self.roads_data.append([w.id,
                                       tag.v,
                                       line])
        except Exception:
            pass    
        
    def area(self, a):
        try:
            wkb = wkbfab.create_multipolygon(a)
            self.count_polygons()
            # load into shapely
            poly = wkblib.loads(wkb, hex=True)
            for tag in a.tags:
                if (tag.k=='building'):
                    self.osm_data.append([a.id,                                   
                                    tag.v,
                                    list(poly.geoms)[0],
                                    list(dict(a.tags).keys()),
                                    dict(a.tags)])
                if (tag.k=='landuse'):
                    self.landuse_data.append([tag.v,
                                    list(poly.geoms)[0],
                                    ])
                if (tag.k == 'amenity'):
                    self.amenity_data.append([tag.v,
                                    list(poly.geoms)[0],
                                    ])
        except Exception:
            pass

In [ ]:
# load official buildings
buildings_path = "./Fairfax_admin_buildings/Fairfax_admin_buildings.shp"
official_buildings = geopandas.read_file(buildings_path)

In [ ]:
official_buildings.columns

In [ ]:
official_buildings = official_buildings.rename(columns={'TYPE': 'official_type'})

In [ ]:
official_buildings.shape

In [ ]:
official_buildings.groupby(['official_type']).count()

In [ ]:
len(official_buildings['OBJECTID'].unique())

In [ ]:
official_buildings.drop(official_buildings[(official_buildings['official_type'] == 'MG') \
                                   | (official_buildings['official_type'] == 'O') \
                                   | (official_buildings['official_type'] == 'MH') \
                                   | (official_buildings['official_type'] == 'MU')].index, inplace = True)

official_buildings.reset_index(inplace=True, drop=True)

In [ ]:
official_buildings.groupby(['official_type']).count()

In [ ]:
official_buildings.shape

In [ ]:
official_buildings['official_type'] = official_buildings['official_type'].apply(\
                                lambda x: 'RES' if (x == 'SFR' \
                                                    or x == 'MFR') else 'NON_RES')

In [ ]:
official_buildings.groupby('official_type').count()

In [ ]:
areahandler = AreaHandler()
areahandler.apply_file("./Datasets/OSM/FFX_County_Buildings.osm.pbf", locations=True)
print("Number of polygons: %d" % areahandler.num_polys)

In [ ]:
len(areahandler.osm_data)

In [ ]:
# transform the list into a pandas DataFrame
data_colnames = ['id', 'building', 'geometry', 'all_tag_keys', 'all_tags']
df = pd.DataFrame(areahandler.osm_data, columns=data_colnames)

In [ ]:
df.head()

In [ ]:
gdf = geopandas.GeoDataFrame(df, crs=4326, geometry='geometry')

In [ ]:
gdf.plot()

In [ ]:
gdf.head()

In [ ]:
gdf.groupby('building').count().sort_values(by=['building'], inplace=False, ascending=True)

In [ ]:
# transform the list into a pandas DataFrame
land_colnames = ['landuse', 'geometry']
land_df = pd.DataFrame(areahandler.landuse_data, columns=land_colnames)

In [ ]:
land_df.shape

In [ ]:
land_df.head()

In [ ]:
land_df.groupby('landuse').count()

In [ ]:
non_res = ['commercial', 'retail','industrial','grass','plant_nursery','quarry','railway','government',\
            'institutional']
land_df['landuse'] = land_df['landuse'].apply(\
                                lambda x: 'non_res' if x in non_res else x)

In [ ]:
land_df.groupby('landuse').count()

In [ ]:
land_gdf = geopandas.GeoDataFrame(land_df, crs=4326, geometry='geometry')

In [ ]:
gdf.shape

In [ ]:
land_gdf.shape

In [ ]:
buildings_land = gdf.sjoin(land_gdf, how="left", predicate='intersects')

In [ ]:
buildings_land.shape

In [ ]:
buildings_land.drop_duplicates(subset='id',inplace=True)

In [ ]:
buildings_land.shape

In [ ]:
buildings_land.head()

In [ ]:
buildings_land.rename(columns={'index_right': 'index_landuse'},inplace=True)
buildings_land['landuse'] = \
            buildings_land['landuse'].apply(lambda x: 'no_landuse' if pd.isnull(x) else x)
buildings_land.reset_index(inplace=True, drop=True)

In [ ]:
buildings_land[buildings_land['landuse'] == 'no_landuse'].shape

In [ ]:
len(buildings_land.index_landuse.unique())

In [ ]:
buildings_land.head()

In [ ]:
buildings_land.groupby('landuse').count()

In [ ]:
buildings_land.groupby(['landuse','building']).count().head(50)

In [ ]:
buildings_land['land_building'] = buildings_land.apply(lambda x: \
    x['landuse'] if x['building'] == 'yes' else x['building'], axis=1)

In [ ]:
buildings_land.groupby(['landuse','land_building']).count().head(50)

In [ ]:
buildings_land.groupby(['landuse','land_building']).count().tail(50)

In [ ]:
def enrich_buildings(buildings_land,tags_list):
    for tag_val in tags_list:
        buildings_land['land_building'] = buildings_land.apply(lambda x: \
            tag_val if x['landuse'] == tag_val else x['land_building'], axis=1)

    return buildings_land

In [ ]:
tags_list = ['residential','commercial','religious','recreation_ground','cemetery',\
            'construction','farmland','farmyard','forest','military']
buildings_land = enrich_buildings(buildings_land,tags_list)

In [ ]:
buildings_land.groupby(['landuse','land_building']).count().head(50)

In [ ]:
buildings_land.groupby(['landuse','land_building']).count().tail(50)

In [ ]:
buildings_land.groupby(['land_building']).count().head(50)

In [ ]:
buildings_land.shape

In [ ]:
official_buildings.shape

In [ ]:
buildings_gdf = buildings_land.sjoin(official_buildings, how="inner", predicate='intersects')
buildings_gdf.rename(columns={'index_right': 'index_official'}, inplace=True)

In [ ]:
buildings_gdf.shape

In [ ]:
buildings_gdf

In [ ]:
buildings_gdf.drop_duplicates(subset='id',inplace=True)

In [ ]:
buildings_gdf.shape

In [ ]:
buildings_gdf.groupby(['landuse','land_building','official_type',]).count().head(50)

In [ ]:
buildings_gdf.groupby(['landuse','land_building','official_type',]).count().tail(50)

In [ ]:
len(areahandler.roads_data)

In [ ]:
# transform the list of roads into a pandas DataFrame
roads_colnames = ['id', 'highway', 'geometry']
roads_df = pd.DataFrame(areahandler.roads_data, columns=roads_colnames)

In [ ]:
roads_df.head()

In [ ]:
roads_df.shape

In [ ]:
road_categories = roads_df.groupby(['highway'])['id'].count().reset_index(name="count")

In [ ]:
road_categories

In [ ]:
roads_cat1 = roads_df[(roads_df['highway'] == 'residential') | (roads_df['highway'] == 'living_street')]

In [ ]:
roads_cat1 = roads_cat1.assign(highway='residential')

In [ ]:
roads_cat1.shape

In [ ]:
roads_cat1.groupby('highway').count()

In [ ]:
roads_cat1 = geopandas.GeoDataFrame(roads_cat1,crs="EPSG:4326",geometry='geometry')

In [ ]:
roads_list = ['primary', 'secondary','tertiary']
roads_cat2 = roads_df[roads_df['highway'].isin(roads_list)]

In [ ]:
roads_cat2 = roads_cat2.assign(highway='non_res_road')

In [ ]:
roads_cat2.shape

In [ ]:
roads_cat2.groupby('highway').count()

In [ ]:
roads_cat2 = geopandas.GeoDataFrame(roads_cat2,crs="EPSG:4326",geometry='geometry')

In [ ]:
roads_cat3 = roads_df[(roads_df['highway'] == 'motorway') | (roads_df['highway'] == 'trunk')]

In [ ]:
roads_cat3 = roads_cat3.assign(highway='roads_cat3')

In [ ]:
roads_cat3.shape

In [ ]:
roads_cat3.groupby('highway').count()

In [ ]:
roads_cat3 = geopandas.GeoDataFrame(roads_cat3,crs="EPSG:4326",geometry='geometry')

In [ ]:
roads_cat4 = roads_df[(roads_df['highway'] == 'service')]

In [ ]:
roads_cat4.shape

In [ ]:
roads_cat4.groupby('highway').count()

In [ ]:
roads_cat4 = geopandas.GeoDataFrame(roads_cat4,crs="EPSG:4326",geometry='geometry')

In [ ]:
def apply_buffers(radius, road_category):
    road_category['geometry'] = road_category['geometry'].to_crs(epsg=32610).buffer(radius)
    road_category.to_crs(epsg=4326, inplace=True)
    road_category.rename(columns={'highway': 'buffered_highway'}, inplace=True)
    road_category = road_category.dissolve()
    
    return road_category

In [ ]:
# approx in meters unit: 1 decimal degree equals 111.32 km 
#radius_list = [0.00027,0.00054,0.00081]
radius_list = [30,60,90]                        # meters
road_category = roads_cat1.copy()
buffered_cat1_30 = apply_buffers(radius_list[0],road_category)
buffered_cat1_60 = apply_buffers(radius_list[1],road_category)
buffered_cat1_90 = apply_buffers(radius_list[2],road_category)

road_category = roads_cat2.copy()
buffered_cat2_30 = apply_buffers(radius_list[0],road_category)
buffered_cat2_60 = apply_buffers(radius_list[1],road_category)
buffered_cat2_90 = apply_buffers(radius_list[2],road_category)

road_category = roads_cat3.copy()
buffered_cat3_30 = apply_buffers(radius_list[0],road_category)
buffered_cat3_60 = apply_buffers(radius_list[1],road_category)
buffered_cat3_90 = apply_buffers(radius_list[2],road_category)

road_category = roads_cat4.copy()
buffered_cat4_30 = apply_buffers(radius_list[0],road_category)
buffered_cat4_60 = apply_buffers(radius_list[1],road_category)
buffered_cat4_90 = apply_buffers(radius_list[2],road_category)

In [ ]:
roads_cat1.plot()
buffered_cat1_30.plot()
buffered_cat1_60.plot()
buffered_cat1_90.plot()

In [ ]:
roads_cat2.plot()
buffered_cat2_30.plot()
buffered_cat2_60.plot()
buffered_cat2_90.plot()

In [ ]:
roads_cat3.plot()
buffered_cat3_30.plot()
buffered_cat3_60.plot()
buffered_cat3_90.plot()

In [ ]:
roads_cat4.plot()
buffered_cat4_30.plot()
buffered_cat4_60.plot()
buffered_cat4_90.plot()

In [ ]:
def join_buildings(buffer_category, roads_category):
    buffered_category = {
        'cat1_30': buffered_cat1_30,
        'cat1_60': buffered_cat1_60,
        'cat1_90': buffered_cat1_90,
        'cat2_30': buffered_cat2_30,
        'cat2_60': buffered_cat2_60,
        'cat2_90': buffered_cat2_90,
        'cat3_30': buffered_cat3_30,
        'cat3_60': buffered_cat3_60,
        'cat3_90': buffered_cat3_90,
        'cat4_30': buffered_cat4_30,
        'cat4_60': buffered_cat4_60,
        'cat4_90': buffered_cat4_90,

    }.get(buffer_category)

    joined = buildings_gdf.sjoin(buffered_category, how='left')
    #joined = buildings_gdf.sjoin(buffered_category, how="left", predicate='intersects')
    joined.rename(columns={'index_right': 'index_roads'}, inplace=True)
    #joined.drop_duplicates(subset='id_left',inplace=True)
    if roads_category:
        joined['buffered_highway'] = \
            joined['buffered_highway'].apply(lambda x: 'not_residential' if pd.isnull(x) else x)
    else:
        joined['buffered_highway'] = \
            joined['buffered_highway'].apply(lambda x: 'residential' if pd.isnull(x) else x)

    joined.reset_index(inplace=True, drop=True)
    
    return joined

In [ ]:
buffer_categories = ['cat1_30', 'cat1_60', 'cat1_90','cat2_30','cat2_60','cat2_90',\
                    'cat3_30', 'cat3_60', 'cat3_90','cat4_30','cat4_60','cat4_90']
roads_category = 1                      # 1 is residential, 0 is ['primary', 'secondary','tertiary']
joined_buildings1 = join_buildings(buffer_categories[0],roads_category)
joined_buildings2 = join_buildings(buffer_categories[1],roads_category)
joined_buildings3 = join_buildings(buffer_categories[2],roads_category)
roads_category = 0
joined_buildings4 = join_buildings(buffer_categories[3],roads_category)
joined_buildings5 = join_buildings(buffer_categories[4],roads_category)
joined_buildings6 = join_buildings(buffer_categories[5],roads_category)
joined_buildings7 = join_buildings(buffer_categories[6],roads_category)
joined_buildings8 = join_buildings(buffer_categories[7],roads_category)
joined_buildings9 = join_buildings(buffer_categories[8],roads_category)
joined_buildings10 = join_buildings(buffer_categories[9],roads_category)
joined_buildings11 = join_buildings(buffer_categories[10],roads_category)
joined_buildings12 = join_buildings(buffer_categories[11],roads_category)

In [ ]:
print(joined_buildings1.shape)
print(joined_buildings2.shape)
print(joined_buildings3.shape)
print(joined_buildings4.shape)
print(joined_buildings5.shape)
print(joined_buildings6.shape)
print(joined_buildings7.shape)
print(joined_buildings8.shape)
print(joined_buildings9.shape)
print(joined_buildings10.shape)
print(joined_buildings11.shape)
print(joined_buildings12.shape)

In [ ]:
joined_buildings1.groupby('official_type').count()

In [ ]:
joined_buildings1.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings1[(joined_buildings1['land_building'] == 'commercial')\
               &(joined_buildings1['buffered_highway'] == 'residential')].shape

In [ ]:
joined_buildings2.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings3.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings4.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings5.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings6.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings7.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings8.groupby(['official_type','buffered_highway']).count()

In [ ]:
joined_buildings9.groupby(['official_type','buffered_highway']).count()

In [ ]:
len(areahandler.osm_data)

In [ ]:
keys_list = []
for sublist in areahandler.osm_data:
    keys_list.append(sublist[3])

In [ ]:
keys_list_flat = [val for sublist in keys_list for val in sublist]

In [ ]:
unique_keys = sorted(list(set(keys_list_flat)))

In [ ]:
unique_keys

In [ ]:
key_freq = []
for key in unique_keys:
    key_freq.append(keys_list_flat.count(key))
    

In [ ]:
len(key_freq)

In [ ]:
keys_tuples = list(zip(unique_keys, key_freq))
keys_freq_df = pd.DataFrame(keys_tuples, columns = ['Key', 'Freq'])

In [ ]:
keys_freq_df.sort_values(by=['Freq'], inplace=True, ascending=False)

In [ ]:
keys_freq_df.reset_index(inplace=True, drop=True)

In [ ]:
keys_freq_df.head()

In [ ]:
keys_freq_df.sort_values(by=['Key'], inplace=False, ascending=True).head(20)

In [ ]:
keys_freq_df.head(21)

In [ ]:
selected_tags = ['source','addr:street','name','building:levels',\
                'roof:shape','amenity','brand','website','shop']
num_keys = len(selected_tags)+2

In [ ]:
len(selected_tags)

In [ ]:
def preprocess_data(input_data, value_tags, selected_tags):
    col_pos = 2
    for tag_key in selected_tags:
        col_name = tag_key
        input_data.insert(col_pos,col_name,0)
        input_data[col_name] = input_data.apply(lambda x: 1 if tag_key in list(x['all_tag_keys']) \
            else 0, axis=1)

    for tag_key in value_tags:
        input_data[tag_key] = input_data.apply(lambda x: x['all_tags'].get(tag_key) \
            if tag_key in list(x['all_tag_keys']) else 0, axis=1)

    # fix anomalies in FFX County data
    input_data.drop(input_data[(input_data['building:levels'] == 'K-6')].index, inplace=True)
    input_data.loc[(input_data['building:levels'] == '-1,1,2,3'),'building:levels']=4

    input_data.drop(columns=['all_tag_keys', 'all_tags'], inplace=True)

    input_data['official_type'] = \
    input_data['official_type'].apply(lambda x: 1 if x == 'RES' else 0)

    return input_data

In [ ]:
def add_buffers(input_data,col_names, buffers_list):
    for num in range(0, len(col_names)):
        input_data.insert(num+2,col_names[num],buffers_list[num])
        input_data[col_names[num]] = input_data[col_names[num]].apply(lambda x: 1 if x == 'residential' else 0)

    return input_data

In [ ]:
#num_buffers = 6
#buffer_loc = 0
#value_tags = ['roof:shape','building:levels']
#col_names = ['buffer_cat1_30','buffer_cat1_60','buffer_cat1_90',\
#            'buffer_cat2_30','buffer_cat2_60','buffer_cat2_90']
#buffers_list = [joined_buildings1['buffered_highway'],joined_buildings2['buffered_highway'],\
#                joined_buildings3['buffered_highway'],joined_buildings4['buffered_highway'],\
#                joined_buildings5['buffered_highway'],joined_buildings6['buffered_highway']]

num_buffers = 12
buffer_loc = 0
value_tags = ['roof:shape','building:levels']
col_names = ['buffer_cat1_30','buffer_cat1_60','buffer_cat1_90',\
            'buffer_cat2_30','buffer_cat2_60','buffer_cat2_90',\
            'buffer_cat3_30','buffer_cat3_60','buffer_cat3_90',\
            'buffer_cat4_30','buffer_cat4_60','buffer_cat4_90']
buffers_list = [joined_buildings1['buffered_highway'],joined_buildings2['buffered_highway'],\
                joined_buildings3['buffered_highway'],joined_buildings4['buffered_highway'],\
                joined_buildings5['buffered_highway'],joined_buildings6['buffered_highway'],\
                joined_buildings7['buffered_highway'],joined_buildings8['buffered_highway'],\
                joined_buildings9['buffered_highway'],joined_buildings10['buffered_highway'],\
                joined_buildings11['buffered_highway'],joined_buildings12['buffered_highway']]
backup_data = joined_buildings1.copy()
#input_data = input_data[['geometry','official_type','land_building','all_tag_keys','all_tags']]
#input_data = input_data[['geometry','official_type','building','landuse','all_tag_keys','all_tags']]
#input_data = input_data[['geometry','official_type','building','landuse',\
#                        'all_tag_keys','all_tags','id_left']]
#input_data = input_data[['geometry','official_type','building','landuse','amenity_key','parking_area',\
#                        'all_tag_keys','all_tags']]
backup_data = backup_data[['geometry','official_type','building','landuse','land_building',\
                        'all_tag_keys','all_tags','id_left']]
backup_data.insert(len(backup_data.columns), 'area',backup_data['geometry'].to_crs(epsg=32610).area)
backup_data = preprocess_data(backup_data,value_tags,selected_tags)

backup_data = add_buffers(backup_data,col_names[buffer_loc:buffer_loc+num_buffers],\
                         buffers_list[buffer_loc:buffer_loc+num_buffers])

In [ ]:
len(areahandler.amenity_data)

In [ ]:
# transform the list of roads into a pandas DataFrame
amenity_colnames = ['amenity_key', 'geometry']
amenity_df = pd.DataFrame(areahandler.amenity_data, columns=amenity_colnames)

In [ ]:
amenity_df.head()

In [ ]:
amenity_df.shape

In [ ]:
amenity_categories = amenity_df.groupby(['amenity_key'])['geometry'].count().reset_index(name="count")

In [ ]:
amenity_categories.tail(50)

In [ ]:
parking_df = amenity_df[(amenity_df['amenity_key'] == 'parking') | (amenity_df['amenity_key'] == 'parking_space')]

In [ ]:
parking_df = parking_df.assign(amenity_key='parking')

In [ ]:
parking_df.reset_index(inplace=True, drop=True)

In [ ]:
parking_df.shape

In [ ]:
parking_gdf = geopandas.GeoDataFrame(parking_df.copy(),crs="EPSG:4326",geometry='geometry')

In [ ]:
parking_gdf.insert(len(parking_gdf.columns), 'parking_area',parking_gdf['geometry'].to_crs(epsg=32610).area)

In [ ]:
parking_gdf.head()

In [ ]:
parking_gdf.shape

In [ ]:
hist = parking_gdf.hist(bins=100, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)
plt.yscale('log')

In [ ]:
breaks = jenkspy.jenks_breaks(parking_gdf['parking_area'], nb_class=3)
print(breaks)

In [ ]:
bins_list = ['bin1','bin2','bin3']
parking_gdf['cut_jenks'] = pd.cut(parking_gdf['parking_area'],
                                bins=breaks,
                                labels=bins_list,
                                include_lowest=True)

In [ ]:
parking_gdf.head()

In [ ]:
for bin in bins_list:
    parking_gdf.insert(len(parking_gdf.columns),bin,0)
    parking_gdf[bin] = parking_gdf.apply(lambda x: 1 if x['cut_jenks'] == bin\
                                     else 0, axis=1)

In [ ]:
parking_gdf.head()

In [ ]:
backup_data.columns

In [ ]:
radius_list = [30,60,90]
counter = 1
for bin in bins_list:
    for radius in radius_list:
        parking_buffer = parking_gdf.copy()
        parking_buffer = parking_buffer[['geometry',bin]]
        col_name = 'parking_cat'+str(counter)
        counter += 1
        parking_buffer.rename(columns={bin: col_name}, inplace=True)
        parking_buffer['geometry'] = parking_buffer['geometry'].to_crs(epsg=32610).buffer(radius)
        parking_buffer.to_crs(epsg=4326, inplace=True)
        buildings_parking = backup_data.sjoin(parking_buffer, how="left", predicate='intersects')
        buildings_parking[col_name] = buildings_parking[col_name].fillna(0)
        buildings_parking.drop_duplicates(subset='id_left',inplace=True)
        buildings_parking.reset_index(inplace=True, drop=True)
        backup_data.insert(len(backup_data.columns),col_name,list(buildings_parking[col_name]))

In [ ]:
backup_data.drop(columns=['id_left'], inplace=True)

In [ ]:
backup_data.shape

In [ ]:
backup_data.head()

In [ ]:
#backup_data.to_pickle("./FFX_Backup_Data/FFX.pkl") 

In [ ]:
#backup_data = pd.read_pickle("./FFX_Backup_Data/FFX.pkl")

In [ ]:
input_data = backup_data.copy()

In [ ]:
input_data.columns

In [ ]:
#input_data.drop(columns=['buffer_cat1_30', 'buffer_cat1_60',\
#                        'buffer_cat1_90', 'buffer_cat2_30', 'buffer_cat2_60', 'buffer_cat2_90',\
#                        'buffer_cat3_30', 'buffer_cat3_60', 'buffer_cat3_90', 'buffer_cat4_30',\
#                        'buffer_cat4_60', 'buffer_cat4_90',\
#                        \
#                        'land_building',\
#                        'parking_cat1', 'parking_cat2', 'parking_cat3', 'parking_cat4',\
#                        'parking_cat5', 'parking_cat6', 'parking_cat7', 'parking_cat8',\
#                        'parking_cat9'], inplace=True)

In [ ]:
input_data.drop(columns=['land_building', 'roof:shape'], inplace=True)

In [ ]:
input_data.columns

In [ ]:
input_data.shape

In [ ]:
input_data.groupby('landuse').count().sort_values(by=['geometry'], inplace=False, ascending=False).head(50)

In [ ]:
input_data.groupby('building').count().sort_values(by=['geometry'], inplace=False, ascending=False).head(50)

In [ ]:
input_data['building'].unique()

In [ ]:
common_buildings = np.array(['apartments', 'church', 'civic', 'commercial', 'construction',
       'detached', 'dormitory', 'garage', 'garages', 'greenhouse',
       'hospital', 'hotel', 'house', 'industrial', 'kindergarten',
       'office', 'parking', 'public', 'residential', 'retail', 'roof',
       'school', 'semidetached_house', 'service', 'shed',
       'static_caravan', 'terrace', 'warehouse', 'yes'], dtype=object)

In [ ]:
unique_buildings = np.setxor1d(input_data['building'].unique(), common_buildings).tolist()

In [ ]:
unique_buildings

In [ ]:
len(unique_buildings)

In [ ]:
input_data[input_data['building'].isin(unique_buildings)].shape

In [ ]:
input_data['building'] = input_data['building'].apply(\
                                lambda x: 'misc_buildings' if x in(unique_buildings)
                                                else x)

In [ ]:
input_data[input_data['building'].isin(unique_buildings)].shape

In [ ]:
input_data.shape

In [ ]:
input_data.groupby('building').count().sort_values(by=['geometry'], inplace=False, ascending=False).head(50)

In [ ]:
common_landuse = np.array(['non_res', 'no_landuse', 'residential', 'construction',
       'forest', 'recreation_ground', 'cemetery', 'farmyard'], dtype=object)

In [ ]:
unique_landuse = np.setxor1d(input_data['landuse'].unique(), common_landuse).tolist()

In [ ]:
unique_landuse

In [ ]:
input_data[input_data['landuse'].isin(unique_landuse)].shape

In [ ]:
input_data['landuse'] = input_data['landuse'].apply(\
                                lambda x: 'misc_landuse' if x in(unique_landuse)
                                                else x)

In [ ]:
input_data[input_data['landuse'].isin(unique_landuse)].shape

In [ ]:
input_data.groupby('landuse').count().sort_values(by=['geometry'], inplace=False, ascending=False).head(50)

In [ ]:
input_data['landuse'].replace(['residential','non_res','forest','cemetery','recreation_ground',\
                                'construction','farmyard'],['residential_landuse','non_res_landuse',\
                                'forest_landuse','cemetery_landuse','recreation_ground_landuse',\
                                'construction_landuse','farmyard_landuse'], inplace=True)

In [ ]:
input_data.groupby('landuse').count().sort_values(by=['geometry'], inplace=False, ascending=False).head(50)

In [ ]:
input_data.shape

In [ ]:
input_data.columns

In [ ]:
FFX_baseline = input_data.copy()

In [ ]:
FFX_baseline.columns

In [ ]:
FFX_baseline = FFX_baseline[['geometry', 'official_type', 'building']]

In [ ]:
FFX_baseline.columns

In [ ]:
FFX_baseline.head()

In [ ]:
residential_list = ['residential','apartments','dormitory', 'house','semidetached_house']
FFX_baseline['building'] = FFX_baseline['building'].apply(\
                                lambda x: 'residential' if x in(residential_list)
                                                else x)

In [ ]:
non_residential_list = ['public','commercial','industrial', 'school','church','office',
                        'retail','hotel','warehouse','kindergarten','civic','hospital']
FFX_baseline['building'] = FFX_baseline['building'].apply(\
                                lambda x: 'non_residential' if x in(non_residential_list)
                                                else x)

In [ ]:
unknown_list = ['yes','detached','terrace', 'garage','roof','shed',
                'parking','garages','greenhouse','static_caravan','service','construction',
                'misc_buildings']
FFX_baseline['building'] = FFX_baseline['building'].apply(\
                                lambda x: 'unknown' if x in(unknown_list)
                                                else x)

In [ ]:
FFX_baseline.head()

In [ ]:
FFX_baseline.groupby('building').count()

In [ ]:
FFX_baseline.groupby(['official_type','building']).count()

In [ ]:
FFX_baseline.to_file("./FFX_Baseline/FFX_Baseline.shp")

In [ ]:
FFX_baseline.groupby('official_type').count()

In [ ]:
#output_filtered.to_file("./Datasets/OSM/Geopandas_Output/filtered_output.shp")
#input_data.to_file("./Datasets/OSM/Geopandas_Output/input_data.shp")

In [ ]:
def nominal_transform(input_data, value_tags):
    ohe = preprocessing.OneHotEncoder(sparse = False)
    for tag in value_tags:
        ohe_results = ohe.fit_transform(input_data[[tag]])
        input_data.reset_index(inplace=True, drop=True)
        input_data = pd.concat([input_data, \
            pd.DataFrame(ohe_results, columns=ohe.categories_[0].tolist())], axis=1)
        input_data.drop(columns=[tag], inplace=True)
        
    return input_data

In [ ]:
# nominal transform

#value_tags = ['roof:shape','building','landuse']
#value_tags = ['roof:shape','land_building']
#value_tags = ['roof:shape','building']
value_tags = ['building','landuse']
nominal_data = input_data.copy()
#nominal_data['roof:shape'] = nominal_data['roof:shape'].astype(str)
nominal_data = nominal_transform(nominal_data, value_tags)
nominal_features = list(nominal_data.columns[2:])
nominal_features

In [ ]:
nominal_data.columns

In [ ]:
sorted(nominal_data.columns)

In [ ]:
nominal_data.shape

In [ ]:
nominal_data.head()

In [ ]:
# latest, building and landuse

X = nominal_data[nominal_features]
y = nominal_data["official_type"]
weights = {0:2.0, 1:1.0}

# Split data into 80% train and 20% test subsets
X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=0.2, shuffle=True)
dt_split = DecisionTreeClassifier(class_weight=weights,min_samples_split=20, \
                                random_state=0,min_impurity_decrease = 0.0001)

#dt_split = DecisionTreeClassifier(min_samples_split=20, random_state=0,min_impurity_decrease = 0.0001)

# Learn on the train subset
dt_split.fit(X_train, y_train)

# Predict the target on the test subset
predicted = dt_split.predict(X_test)

# 1 = RES; 0 = NON_RES

disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

print(
    f"Classification report for classifier {dt_split}:\n"
    f"{metrics.classification_report(y_test, predicted)}\n"
)

In [ ]:
# result for generating output shape file

X = nominal_data[nominal_features]
y = nominal_data["official_type"]
#weights = 'balanced'
weights = {0:2.0, 1:1.0}
dt = DecisionTreeClassifier(class_weight=weights,min_samples_split=20, random_state=0,min_impurity_decrease = 0.0001)
y_pred = cross_val_predict(dt, X, y, cv=10)
disp = metrics.ConfusionMatrixDisplay.from_predictions(y, y_pred)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

print(
    f"Classification report for classifier {dt}:\n"
    f"{metrics.classification_report(y, y_pred)}\n"
)

In [ ]:
output_data = nominal_data.copy()
output_data.drop(columns=['official_type','buffer_cat2_30', 'buffer_cat2_60','buffer_cat2_90'], inplace=True)
output_data.insert(1,'official',y)
output_data.insert(2,'predicted',y_pred)
output_filtered = output_data.copy()
#output_filtered = output_filtered[output_filtered['official'] != output_filtered['predicted']]
output_filtered['official'] = \
    output_filtered['official'].apply(lambda x: 'RES' if x == 1 else 'NON_RES')
output_filtered['predicted'] = \
    output_filtered['predicted'].apply(lambda x: 'RES' if x == 1 else 'NON_RES')

In [ ]:
#output_filtered.to_file("./Datasets/OSM/Geopandas_Output/filtered_output.shp")
output_data.to_file("./FFX_Predictions/FFX_Predictions.shp")

In [ ]:
# Test on Mecklenburg 20% data

Mecklenburg_nominal = pd.read_pickle("./Mecklenburg_Processed_Data/Mecklenburg_nominal_data.pkl")

X = nominal_data[nominal_features]
y = nominal_data["official_type"]

Mecklenburg_X = Mecklenburg_nominal[nominal_features]
Mecklenburg_y = Mecklenburg_nominal["official_type"]

# Split data into 80% train and 20% test subsets
X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=0.2, shuffle=True)

Mecklenburg_X_train, Mecklenburg_X_test, Mecklenburg_y_train, Mecklenburg_y_test = train_test_split(\
    Mecklenburg_X, Mecklenburg_y, test_size=0.2, shuffle=True)

#weights = 'balanced'
weights = {0:2.0, 1:1.0}
dt_split = DecisionTreeClassifier(class_weight=weights,min_samples_split=20, \
                                random_state=0,min_impurity_decrease = 0.0001)

#dt_split = DecisionTreeClassifier(min_samples_split=20, random_state=0,min_impurity_decrease = 0.0001)

# Learn on the train subset
dt_split.fit(X_train, y_train)

# Predict the target on the test subset
predicted = dt_split.predict(Mecklenburg_X_test)

# 1 = RES; 0 = NON_RES

disp = metrics.ConfusionMatrixDisplay.from_predictions(Mecklenburg_y_test, predicted)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

print(
    f"Classification report for classifier {dt_split}:\n"
    f"{metrics.classification_report(Mecklenburg_y_test, predicted)}\n"
)

In [ ]:
# Test on Mecklenburg 100% data

Mecklenburg_nominal = pd.read_pickle("./Mecklenburg_Processed_Data/Mecklenburg_nominal_data.pkl")

X = nominal_data[nominal_features]
y = nominal_data["official_type"]

Mecklenburg_X = Mecklenburg_nominal[nominal_features]
Mecklenburg_y = Mecklenburg_nominal["official_type"]

#weights = 'balanced'
weights = {0:2.0, 1:1.0}
dt_split = DecisionTreeClassifier(class_weight=weights,min_samples_split=20, \
                                random_state=0,min_impurity_decrease = 0.0001)

X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=0.2, shuffle=True)

# Learn on FFX
dt_split.fit(X_train, y_train)

# Predict the target on Mecklenburg
predicted = dt_split.predict(Mecklenburg_X)

# 1 = RES; 0 = NON_RES

disp = metrics.ConfusionMatrixDisplay.from_predictions(Mecklenburg_y, predicted)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

print(
    f"Classification report for classifier {dt_split}:\n"
    f"{metrics.classification_report(Mecklenburg_y, predicted)}\n"
)

In [ ]:
output_data = Mecklenburg_nominal.copy()
output_data.drop(columns=['official_type','buffer_cat2_30', 'buffer_cat2_60','buffer_cat2_90'], inplace=True)
output_data.insert(1,'official',Mecklenburg_y)
output_data.insert(2,'predicted',predicted)

In [ ]:
output_data.head()

In [ ]:
#output_filtered.to_file("./Datasets/OSM/Geopandas_Output/filtered_output.shp")
output_data.to_file("./Test_on_Mecklenburg/Mecklenburg_Test_Predictions.shp")

In [ ]:
# Test on Boulder 20% data

Boulder_nominal = pd.read_pickle("./Boulder_Processed_Data/Boulder_nominal_data.pkl")

X = nominal_data[nominal_features]
y = nominal_data["official_type"]

#Boulder_X = pd.read_pickle("./Boulder_Processed_Data/Boulder_X.pkl")
#Boulder_y = pd.read_pickle("./Boulder_Processed_Data/Boulder_y.pkl")

Boulder_X = Boulder_nominal[nominal_features]
Boulder_y = Boulder_nominal["official_type"]

# Split data into 80% train and 20% test subsets
X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=0.2, shuffle=True)

Boulder_X_train, Boulder_X_test, Boulder_y_train, Boulder_y_test = train_test_split(\
    Boulder_X, Boulder_y, test_size=0.2, shuffle=True)

#weights = 'balanced'
weights = {0:2.0, 1:1.0}
dt_split = DecisionTreeClassifier(class_weight=weights,min_samples_split=20, \
                                random_state=0,min_impurity_decrease = 0.0001)

#dt_split = DecisionTreeClassifier(min_samples_split=20, random_state=0,min_impurity_decrease = 0.0001)

# Learn on the train subset
dt_split.fit(X_train, y_train)

# Predict the target on the test subset
predicted = dt_split.predict(Boulder_X_test)

# 1 = RES; 0 = NON_RES

disp = metrics.ConfusionMatrixDisplay.from_predictions(Boulder_y_test, predicted)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

print(
    f"Classification report for classifier {dt_split}:\n"
    f"{metrics.classification_report(Boulder_y_test, predicted)}\n"
)

In [ ]:
# Test on Boulder 100% data

Boulder_nominal = pd.read_pickle("./Boulder_Processed_Data/Boulder_nominal_data.pkl")

X = nominal_data[nominal_features]
y = nominal_data["official_type"]

Boulder_X = Boulder_nominal[nominal_features]
Boulder_y = Boulder_nominal["official_type"]

#weights = 'balanced'
weights = {0:2.0, 1:1.0}
dt_split = DecisionTreeClassifier(class_weight=weights,min_samples_split=20, \
                                random_state=0,min_impurity_decrease = 0.0001)

X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=0.2, shuffle=True)

# Learn on FFX
dt_split.fit(X_train, y_train)

# Predict the target on Boulder
predicted = dt_split.predict(Boulder_X)

# 1 = RES; 0 = NON_RES

disp = metrics.ConfusionMatrixDisplay.from_predictions(Boulder_y, predicted)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

print(
    f"Classification report for classifier {dt_split}:\n"
    f"{metrics.classification_report(Boulder_y, predicted)}\n"
)

In [ ]:
output_data = Boulder_nominal.copy()
output_data.drop(columns=['official_type','buffer_cat2_30', 'buffer_cat2_60','buffer_cat2_90'], inplace=True)
output_data.insert(1,'official',Boulder_y)
output_data.insert(2,'predicted',predicted)

In [ ]:
output_data.head()

In [ ]:
#output_filtered.to_file("./Datasets/OSM/Geopandas_Output/filtered_output.shp")
output_data.to_file("./Test_on_Boulder/Boulder_Test_Predictions.shp")

In [ ]:
#X.to_pickle("./FFX_Processed_Data/FFX_X.pkl") 
#y.to_pickle("./FFX_Processed_Data/FFX_y.pkl")
nominal_data.to_pickle("./FFX_Processed_Data/FFX_nominal_data.pkl") 

In [ ]:
# Ablation Study

In [ ]:
input_data.columns

In [ ]:
ablation_data = input_data.copy()

In [ ]:
ablation_data.drop(columns=['buffer_cat1_30', 'buffer_cat1_60',\
                        'buffer_cat1_90', 'buffer_cat2_30', 'buffer_cat2_60', 'buffer_cat2_90',\
                        'buffer_cat3_30', 'buffer_cat3_60', 'buffer_cat3_90', 'buffer_cat4_30',\
                        'buffer_cat4_60', 'buffer_cat4_90',\
                        \
                        \
                        'parking_cat1', 'parking_cat2', 'parking_cat3', 'parking_cat4',\
                        'parking_cat5', 'parking_cat6', 'parking_cat7', 'parking_cat8',\
                        'parking_cat9'], inplace=True)

In [ ]:
ablation_data.columns

In [ ]:
# nominal transform

value_tags = ['building','landuse']
#nominal_data = input_data.copy()
#value_tags = ['building']
ablation_data = nominal_transform(ablation_data, value_tags)
ablation_features = list(ablation_data.columns[2:])
ablation_features

In [ ]:
ablation_data.columns

In [ ]:
ablation_data.shape

In [ ]:
X = ablation_data[ablation_features]
y = ablation_data["official_type"]
weights = {0:2.0, 1:1.0}

# Split data into 80% train and 20% test subsets
X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=0.2, shuffle=True)
dt_split = DecisionTreeClassifier(class_weight=weights,min_samples_split=20, \
                                random_state=0,min_impurity_decrease = 0.0001)

#dt_split = DecisionTreeClassifier(min_samples_split=20, random_state=0,min_impurity_decrease = 0.0001)

# Learn on the train subset
dt_split.fit(X_train, y_train)

# Predict the target on the test subset
predicted = dt_split.predict(X_test)

# 1 = RES; 0 = NON_RES

disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

print(
    f"Classification report for classifier {dt_split}:\n"
    f"{metrics.classification_report(y_test, predicted)}\n"
)